In [1]:
import os
import numpy as np
import pandas as pd
import sqlite3 as sql

MPC_DATA_DIR = "mpc_data"
UOD = "/mnt/data/projects/precovery/precovery_data"
FRAME_DB = os.path.join(UOD, "index.db")
! du -sh {UOD}
! du -sh {FRAME_DB}

os.environ["OORB_DATA"] = "/home/moeyensj/software/oorb/share/oorb/"

300G	/mnt/data/projects/precovery/precovery_data
888M	/mnt/data/projects/precovery/precovery_data/index.db


In [2]:
MPCORB_FILE = os.path.join(MPC_DATA_DIR, "mpcorb_extended.json")
if not os.path.exists(MPCORB_FILE):
    ! wget https://minorplanetcenter.net/Extended_Files/mpcorb_extended.json.gz
    ! gunzip mpcorb_extended.json.gz
    ! cp mpcorb_extended.json {MPC_DATA_DIR}/mpcorb_extended.json
    ! rm mpcorb_extended.json.gz
    pass

In [3]:
from thor.orbits import Orbits
from thor.utils.mpc import readMPCOrbitCatalog

mpc_orbits_df = readMPCOrbitCatalog(MPCORB_FILE)
print(f"Total MPC orbits: {len(mpc_orbits_df)}")

# Filter the MPC orbits to only contain those observed since 2010
year_last_observed = mpc_orbits_df["last_obs"].str[:4].astype(int)
mpc_orbits_df = mpc_orbits_df[year_last_observed >= 2010]
print(f"MPC orbits observed since 2010: {len(mpc_orbits_df)}")

mpc_orbits = Orbits.fromMPCOrbitCatalog(mpc_orbits_df)
mpc_orbits.keplerian
mpc_orbits.cartesian

Total MPC orbits: 1298457
MPC orbits observed since 2010: 1291106


array([[-2.50315269e+00,  2.64980455e-01,  4.69470013e-01,
        -1.47228816e-03, -1.10463649e-02, -7.80683705e-05],
       [-1.11238330e+00,  1.53946174e+00, -9.71053802e-01,
        -1.10274737e-02, -5.27296083e-03,  4.60346390e-03],
       [ 1.44565128e+00,  1.33085314e+00, -3.61024533e-01,
        -9.86783949e-03,  9.22837891e-03, -1.69414305e-03],
       ...,
       [-1.92119886e+00,  5.04558475e-01,  2.25208855e-02,
        -2.54295007e-03, -1.02969229e-02,  4.71847442e-03],
       [-1.31287291e+00,  5.90954243e-01, -4.22139862e-01,
        -6.90281154e-03, -8.07382729e-03,  1.20935143e-02],
       [-1.46891458e+00, -6.52424566e-02,  6.66683163e-01,
         3.72923655e-03, -1.45222657e-02,  2.21371533e-03]])

In [4]:
con = sql.connect(FRAME_DB)
frames = pd.read_sql("""SELECT * FROM frames WHERE dataset_id = 'nsc'""", con)
frames.insert(2, "month", frames["data_uri"].str.split("/").str[1])
con.close()

In [5]:
months = frames["month"].unique()

In [6]:
from precovery.precovery_db import PrecoveryDatabase
import dataclasses

def get_observations_for_month(month):

    db = PrecoveryDatabase.from_dir(UOD, create=False, mode="r", allow_version_mismatch=True)
    con = sql.connect(FRAME_DB)
    frames_window = frames[frames["month"] == month]
    ids = frames_window["id"].values.tolist()
    frames_window = list(db.frames.idx.get_frames_by_id(ids))

    observation_dfs = []

    for f in frames_window:
        mjds = []
        ras = []
        decs = []
        ra_sigmas = []
        dec_sigmas = []
        mags = []
        mag_sigmas = []
        ids = []
        for o in db.frames.iterate_observations(f):
            mjds.append(o.mjd)
            ras.append(o.ra)
            decs.append(o.dec)
            ra_sigmas.append(o.ra_sigma)
            dec_sigmas.append(o.dec_sigma)
            mags.append(o.mag)
            mag_sigmas.append(o.mag_sigma)
            ids.append(o.id.decode())

        observations_df = pd.DataFrame({
            "obs_id" : ids,
            "mjd_utc": mjds,
            "ra": ras,
            "dec": decs,
            "ra_sigma": ra_sigmas,
            "dec_sigma": dec_sigmas,
            "mag": mags,
            "mag_sigma": mag_sigmas
        })
        for i, (key, value) in enumerate(dataclasses.asdict(f).items()):
            if type(value) == b'':
                value = value.decode()
            observations_df.insert(i, key, value)
        observation_dfs.append(observations_df)

    observations = pd.concat(observation_dfs, ignore_index=True)
    observations.rename(columns={"id": "frame_id"}, inplace=True)
    observations.drop(columns=["dataset_id", "data_uri", "data_offset", "data_length"], inplace=True)
    observations.sort_values(by=["mjd_utc"], inplace=True, ignore_index=True)

    con.close()
    return observations

In [7]:
from astropy.time import Time
import numpy as np
import healpy as hp
import warnings
import logging
from thor import selectTestOrbits
from thor import preprocessObservations
from thor import rangeAndShift
from thor.orbits import propagateOrbits
from thor.orbits import generateEphemeris

orbits = []

logger = logging.getLogger("thor")
logger.setLevel(logging.CRITICAL)

np.random.seed(42)

for i, month in enumerate(months):

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")

        if not os.path.exists(f"test_orbits_{month}.parquet"):
            # Read the observations for the month
            observations = get_observations_for_month(month)

            # Calculate the middle point of the month
            mjd_middle_point = (observations["mjd_utc"].max() - observations["mjd_utc"].min()) / 2 + observations["mjd_utc"].min()
            delta_middle_point = np.abs(observations["mjd_utc"].values - mjd_middle_point)
            closest_to_middle_idx = np.argsort(delta_middle_point)[0]
            middle_point = Time(
                observations["mjd_utc"].values[closest_to_middle_idx:closest_to_middle_idx+1],
                scale="utc", 
                format="mjd"
            )

            if i > 0:
                # Load propagated orbits from the previous month to reduce propagation time
                file_name = os.path.join(MPC_DATA_DIR, f"mpcorb_propagated_{months[i-1]}.parquet")
                mpcorb_propagated = pd.read_parquet(file_name)
                mpc_orbits = Orbits.from_df(mpcorb_propagated)

            # Propagate the MPC orbit catalog to the middle point of the month
            file_name = os.path.join(MPC_DATA_DIR, f"mpcorb_propagated_{month}.parquet")
            if not os.path.exists(file_name):
                mpcorb_propagated = propagateOrbits(
                    mpc_orbits,
                    middle_point,
                    num_jobs=60,
                    parallel_backend="mp",
                    chunk_size=100,
                    backend="PYOORB",
                    backend_kwargs={"dynamical_model" : "N"}
                )
                mpcorb_propagated.to_parquet(file_name)
                
            else:
                mpcorb_propagated = pd.read_parquet(file_name)
            print("Orbits propagated: ", len(mpcorb_propagated))

            # Generate the ephemeris for the middle point of the month for the MPC orbit catalog
            observers = {
                "W84" : middle_point
            }
            file_name = os.path.join(MPC_DATA_DIR, f"mpcorb_ephemeris_{month}.parquet")
            if not os.path.exists(file_name):
                mpcorb_eph = generateEphemeris(
                    Orbits.from_df(mpcorb_propagated),
                    observers,
                    num_jobs=60,
                    parallel_backend="mp",
                    chunk_size=100,
                    backend="PYOORB",
                    backend_kwargs={"dynamical_model" : "N"}
                )
                mpcorb_eph.to_parquet(file_name)
                
            else:
                mpcorb_eph = pd.read_parquet(file_name)
            print("Ephemerides generated: ", len(mpcorb_eph))

            # Preprocess the observations into the format required by THOR
            column_mapping = {
                "obs_id" : "obs_id",
                "mjd" : "mjd_utc",
                "RA_deg" : "ra",
                "Dec_deg" : "dec",
                "RA_sigma_deg" : "ra_sigma",
                "Dec_sigma_deg" : "dec_sigma",
                "observatory_code" : "obscode",
                "obj_id" : None,
                "mag" : "mag",
                "mag_sigma" : "mag_sigma",
                "filter" : "filter",
            }   
            astrometric_errors = None
            mjd_scale = "utc"

            preprocessed_observations, preprocessed_associations = preprocessObservations(
                observations,
                column_mapping,
                mjd_scale=mjd_scale,
                astrometric_errors=astrometric_errors
            )

            NSIDE = 32
            test_orbits_df = selectTestOrbits(
                preprocessed_observations, 
                mpcorb_eph, 
                patch_algorithm="healpix", 
                patch_algorithm_kwargs={
                    "nside" : NSIDE, 
                    "nest" : True
                }
            )
            
            # Select two patches at random
            patches = test_orbits_df["patch_id"].unique()
            patch_ids = np.random.choice(patches, 1)
            patches_selected = test_orbits_df[test_orbits_df["patch_id"].isin(patch_ids)].reset_index(drop=True)

            # For each orbit in the select patches, project the observations
            # and calculate the number of exposures and observations
            orbits_month = []
            for orbit in patches_selected["orbit_id"].unique():

                orbit_df = patches_selected[patches_selected["orbit_id"] == orbit]
                projected_observations = rangeAndShift(
                    preprocessed_observations,
                    Orbits.from_df(orbit_df),
                    num_jobs=20,
                    parallel_backend="mp",
                    cell_area=10,
                )

                num_exposures = projected_observations["mjd_utc"].nunique()
                num_obs = len(projected_observations)

                orbit_df.insert(0, "month", month)
                orbit_df.insert(3, "num_exposures", num_exposures)
                orbit_df.insert(4, "num_obs", num_obs)

                orbits.append(orbit_df)
                orbits_month.append(orbit_df)

            orbits_month = pd.concat(orbits_month, ignore_index=True)
            orbits_month.to_parquet(f"test_orbits_{month}.parquet")

    

orbits = pd.concat(orbits, ignore_index=True)
orbits.to_parquet("test_orbits.parquet")

Running version: 0.2.dev160+g9988cf8
Database version: 0.2.dev159+gc091c7b
allow_version_mismatch=True, so continuing.
